#Traitement des données de démographie

In [ ]:
demographie

In [ ]:
#Choisissons les dates auxquelles il y'a eu des élections
colonne_a_gardé=['LIBGEO','PSDC1999']
Demographie =demographie[colonne_a_gardé].copy()
for i in ['PMUN2007','PMUN2012','PMUN2017','PMUN2021']:
    colonne=['LIBGEO',i]
    tableau = demographie[colonne]
    tableau.columns=colonne_a_gardé
    Demographie = pd.concat([Demographie, tableau], ignore_index=True)
Demographie.columns=['Libellé de la commune','Population']
Demographie['Date']=[2002] * 34947 + [2007] * 34947 + [2012] * 34947+ [2017] * 34947 + [2022] *34947
Demographie

#Traitement des données d'education et generation des echantillons pour chaque commune

In [ ]:
import pandas as pd

# Chemin vers votre fichier Excel
chemin_fichier = r"C:\Users\sebeo\OneDrive\Bureau\DATASET\donnés d'education.xlsx"
# Lire le fichier Excel
education = pd.read_excel(chemin_fichier)

# Filtrons les lignes pour les années spécifiées auquels il y'a eu des elections
annees_a_garder = [2002, 2007, 2012, 2017, 2022]
df_filtre = education[education['Année'].isin(annees_a_garder)]

# Réinitialiser l'index
df_filtre = df_filtre.reset_index(drop=True)

# Afficher le DataFrame filtré
df_filtre

In [ ]:
def generate_normal_samples(mu_values, sigma_values, size):
    """
    Génère des échantillons suivant une distribution normale.

    Args:
        mu_values (float): Moyenne de la distribution normale.
        sigma_values (float): Écart-type de la distribution normale.
        size (int or tuple of ints): Taille de l'échantillon à générer.

    Returns:
        np.ndarray: Échantillons suivant une distribution normale.
    """
    samples = np.random.normal(loc=mu_values, scale=sigma_values, size=size)
    return samples

*pour generer les taux de reussite en se basant sur le nombre de personnes dans chaque commune et en supposant que la distribution suit une loi normal, nous allons, tout abord determiner les poids qui seront affecter à chaque commune afin d'avoir une génération de données représentative de la commune en question*

In [ ]:
Dates=[2002,2007,2012,2017,2022]
listes_des_poids=[]

for date in Dates:
  # Filtrer les lignes où l'année est égale à 2002
  population = Demographie[Demographie['Date'] == date]

 # Calculer la somme de la population pour l'année 2002
  somme_population = population['Population'].sum()

  listes_des_poids.extend(population['Population']/somme_population)


In [ ]:
listes_des_poids

In [ ]:
import numpy as np

# Créer une liste pour stocker les échantillons générés
Bac_Tech_list = []
Bac_Gen_list = []
Bac_Pro_list = []

for i in range(5):
    # Calculer l'écart-type comme la racine carrée de la variance
    sigma_values = 1
    size = 34947

    # Utiliser sigma_values pour générer des échantillons suivant une loi normale
    Bac_Tech = generate_normal_samples(df_filtre['Baccalauréat technologique'][i], sigma_values, size)
    Bac_Gen = generate_normal_samples(df_filtre['Baccalauréat général'][i], sigma_values, size)
    Bac_Pro = generate_normal_samples(df_filtre['Baccalauréat professionnel'][i], sigma_values, size)

    # Ajouter les échantillons générés à la liste
    Bac_Tech_list.extend(Bac_Tech)
    Bac_Gen_list.extend(Bac_Gen)
    Bac_Pro_list.extend(Bac_Pro)

# Ajuster les échantillons en fonction de la population de chaque commune
Bac_Tech_list_weighted = [Bac_Tech_list[i] * listes_des_poids[i] for i in range(len(Bac_Tech_list))]
Bac_Gen_list_weighted = [Bac_Gen_list[i] * listes_des_poids[i] for i in range(len(Bac_Gen_list))]
Bac_Pro_list_weighted = [Bac_Pro_list[i] * listes_des_poids[i] for i in range(len(Bac_Pro_list))]

# Créer une liste d'années pondérées pour chaque échantillon
Année = [df_filtre['Année'][i] for i in range(len(df_filtre['Année']))] * 34947

# Créer le DataFrame en utilisant les listes de données ajustées
Niveau_education = pd.DataFrame({
    'Libellé de la commune':Demographie['Libellé de la commune'],
    'Baccalauréat général': Bac_Gen_list_weighted,
    'Baccalauréat technologique': Bac_Tech_list_weighted,
    'Baccalauréat professionnel': Bac_Pro_list_weighted,
    'Date': Année
})

In [ ]:
Niveau_education

# Traitement des Données de revenu des menages par communes

 *On constate qu'a apartir de la colonne MED12 LE nombre d'elements manquants est très elevé; ce qui nous amene a ne garder que les 5 premieres colonnes*

In [ ]:
import pandas as pd
import os
from tqdm import tqdm

# Liste des noms de fichiers Excel
fichiers_excel = [
    r"C:\Users\sebeo\OneDrive\Bureau\DATASET\revenu\revenu 2012.xlsx",
    r"C:\Users\sebeo\OneDrive\Bureau\DATASET\revenu\revenu 2017.xlsx",
    r"C:\Users\sebeo\OneDrive\Bureau\DATASET\revenu\revenu 2022.xlsx",
]

# Initialisation d'une liste pour stocker les nouveaux DataFrames et les dates correspondantes
revenu_dataframes = []
dates = [2012, 2017, 2022]

# Parcourir chaque fichier Excel
for fichier, date in zip(fichiers_excel, dates):

    # Lire le fichier Excel
    revenu = pd.read_excel(fichier)

    # Sélectionner les cinq premières colonnes
    revenu = revenu.iloc[:, :5]
    revenu.columns=['CODGEO', 'LIBGEO', 'NBMENFISC','NBPERSMENFISC12','MED']

    # Exclure la colonne 'LIBGEO' du processus
    for col in revenu.columns:
        if col != 'LIBGEO':
            # Conversion de la colonne en numérique, en remplaçant les valeurs non numériques par NaN
            revenu[col] = pd.to_numeric(revenu[col], errors='coerce')

            # Calcul de la médiane des valeurs numériques dans la colonne
            median_value = revenu[col].median()

            # Remplacement des valeurs non numériques par la médiane calculée
            revenu[col].fillna(median_value, inplace=True)

    # Ajout de la colonne 'Date' avec une année donnée
    revenu['Date'] = date

    # Ajouter le DataFrame au liste
    revenu_dataframes.append(revenu)

# Concaténer tous les DataFrames en un seul
revenu_final = pd.concat(revenu_dataframes, ignore_index=True)

revenu_final.columns=['CODGEO', 'Libellé de la commune', 'NBMENFISC','NBPERSMENFISC12','MED', 'Date']

# Afficher les premières lignes du DataFrame final
revenu_final



*   'CODGEO' : le code associer à chaque commune
*  'NBMENFISC' : Nombre de ménages fiscaux

*   'NBPERSMENFISC12' : Nombre de personnes dans les ménages fiscaux
*   'MED' : Médiane du niveau de vie (€)




# Visualisation et interprétations

In [ ]:
# Grouper les données par le nom de la commune et calculer la somme des populations
communes_grouped = Demographie.groupby('Libellé de la commune')['Population'].sum()

# Trier les communes par ordre décroissant de population et sélectionner les cinq premières
top_5_communes = communes_grouped.sort_values(ascending=False).head(5)

# Afficher les cinq communes les plus peuplées
print(top_5_communes)



In [ ]:
import matplotlib.pyplot as plt

# Liste des cinq communes les plus peuplées
top_5_communes_list = top_5_communes.index.tolist()

# Filtrer les données pour les cinq communes les plus peuplées
top_5_communes_data = Demographie[Demographie['Libellé de la commune'].isin(top_5_communes_list)]

# Création d'une figure et des axes pour les sous-graphiques
fig, axs = plt.subplots(5, 1, figsize=(10, 20), sharex=True)

# Parcours des cinq communes
for i, commune in enumerate(top_5_communes_list):
    # Sélection des données pour la commune spécifique
    commune_data = top_5_communes_data[top_5_communes_data['Libellé de la commune'] == commune]
    # Tracé de l'histogramme de la population pour chaque année
    axs[i].bar(commune_data['Date'], commune_data['Population'])
    # Ajout de titre pour chaque sous-graphique
    axs[i].set_title(commune)
    axs[i].set_ylabel('Population')
    axs[i].set_xlabel('Date')

# Ajout de titre pour l'ensemble du graphique
fig.suptitle('Évolution de la population des cinq communes les plus peuplées (2002-2022)')

# Affichage du graphique
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Liste des années
annees = [2012, 2017, 2022]

# Liste des cinq communes
top_5_communes_list = ['Marseille', 'Lyon', 'Toulouse', 'Nice', 'Nantes']

# Création d'une figure et des axes pour les sous-graphiques
fig, axs = plt.subplots(5, len(annees), figsize=(20, 20), sharex=True, sharey=True)

# Parcours des années
for j, annee in enumerate(annees):
    # Parcours des cinq communes
    for i, commune in enumerate(top_5_communes_list):
        # Sélection des données pour la commune spécifique et l'année spécifique
        commune_data = resultat_final[(resultat_final['Libellé de la commune'] == commune) & (resultat_final['Date'] == annee)]
        # Regroupement des données par candidat et calcul du nombre total de voix pour chaque candidat
        votes_par_candidat = commune_data.groupby('Nom')['Voix'].sum()
        # Création du diagramme circulaire
        axs[i, j].pie(votes_par_candidat, labels=votes_par_candidat.index, autopct='%1.1f%%', startangle=140)
        # Ajout de titre pour chaque sous-graphique
        axs[i, j].set_title(f"{commune} - {annee}")

# Ajout de titre pour l'ensemble du graphique
fig.suptitle('Distribution des votes par candidat dans les cinq communes les plus peuplées (2012, 2017, 2022)')

# Ajustement de la disposition des sous-graphiques
plt.tight_layout()

# Affichage du graphique
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Liste des années
annees = [2012, 2017, 2022]

# Colonnes à tracer
colonnes = ['NBMENFISC', 'NBPERSMENFISC12', 'MED']

# Libellés des communes spécifiques
communes_specifiques = ['Marseille', 'Lyon', 'Toulouse', 'Nice', 'Nantes']

# Boucle sur les libellés de commune spécifiques
for commune in communes_specifiques:
    # Création d'une figure et d'axes pour chaque commune
    fig, ax = plt.subplots(figsize=(12, 8))

    # Titre du graphique avec le nom de la commune
    ax.set_title(f'Évolution des différentes colonnes de revenu pour la commune de {commune}')

    # Boucle sur les colonnes
    for i, colonne in enumerate(colonnes):
        # Initialisation des données pour la colonne
        donnees_colonne = []
        # Parcours des années
        for annee in annees:
            # Sélection des données pour l'année spécifique, la colonne spécifique et le libellé de commune spécifique
            annee_data = revenu_final[(revenu_final['Date'] == annee) & (revenu_final['Libellé de la commune'] == commune)]
            valeurs_colonne = annee_data[colonne]
            # Calcul de la moyenne pour l'année spécifique
            moyenne_valeurs = np.mean(valeurs_colonne)
            # Ajout de la moyenne calculée aux données de la colonne
            donnees_colonne.append(moyenne_valeurs)
        # Tracé de la courbe pour la colonne
        ax.plot(annees, donnees_colonne, label=colonne)

    # Étiquettes pour les axes
    ax.set_xlabel('Année')
    ax.set_ylabel('Valeur moyenne')

    # Légende
    ax.legend()

    # Affichage du graphique pour la commune spécifique
    plt.grid(True)
    plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np


# Colonnes à tracer
colonnes = ['Baccalauréat général', 'Baccalauréat technologique', 'Baccalauréat professionnel']

# Libellés des communes spécifiques
communes_specifiques = ['Marseille', 'Lyon', 'Toulouse', 'Nice', 'Nantes']

# Création d'une figure et d'axes
fig, axs = plt.subplots(len(communes_specifiques), len(colonnes), figsize=(15, 15))

# Boucle sur les libellés de commune spécifiques
for i, commune in enumerate(communes_specifiques):
    # Sélection des données pour la commune spécifique
    commune_data = Niveau_education[Niveau_education['Libellé de la commune'] == commune]

    # Boucle sur les colonnes
    for j, colonne in enumerate(colonnes):
        # Données à tracer
        donnees = commune_data[colonne]

        # Tracé de l'histogramme
        axs[i, j].hist(donnees, bins=10, color='skyblue', edgecolor='black')

        # Ajout de titres et d'étiquettes
        axs[i, j].set_title(f'{commune} - {colonne}')
        axs[i, j].set_xlabel('Taux de réussite')
        axs[i, j].set_ylabel('Fréquence')
        axs[i, j].grid(True)

# Ajustement de l'espacement entre les sous-graphiques
plt.tight_layout()

# Affichage du graphique
plt.show()


#traitement des données d'elections

traitement des données du premier tour

In [ ]:
import pandas as pd
import os
from tqdm import tqdm

# Liste des noms de fichiers Excel
fichiers_excel = [
    r"C:\Users\sebeo\OneDrive\Bureau\DATASET\tr 1\2002 tr 1.xls",
    r"C:\Users\sebeo\OneDrive\Bureau\DATASET\tr 1\2007 tr 1.xlsx",
    r"C:\Users\sebeo\OneDrive\Bureau\DATASET\tr 1\2012 tr 1.xlsx",
    r"C:\Users\sebeo\OneDrive\Bureau\DATASET\tr 1\2017 tr 1.xls",
    r"C:\Users\sebeo\OneDrive\Bureau\DATASET\tr 1\2022 tr 1.xls"
]

# Initialisation d'une liste pour stocker les nouveaux DataFrames et les dates correspondantes
nouveaux_dataframes = []
dates = [2002,2007,2012,2017,2022]

# Parcourir chaque fichier Excel
for fichier, date in zip(fichiers_excel, dates):

    # Charger le fichier Excel dans un DataFrame pandas
    df = pd.read_excel(fichier)

    # Colonnes à garder
    colonnes_a_garder = ['Libellé de la commune','Inscrits','Abstentions','Votants','Exprimés', 'Sexe', 'Nom', 'Prénom', 'Voix', '% Voix/Ins', '% Voix/Exp']

    # Création d'un nouveau DataFrame avec les colonnes sélectionnées
    nouveau_df = df[colonnes_a_garder].copy()

    # Ajout des valeurs des colonnes supplémentaires
    x = int((df.shape[1] - 13) / 6)
    for j in tqdm(range(1, x)):
        colonnes = ['Libellé de la commune','Inscrits','Abstentions','Votants','Exprimés', f'Sexe.{j}', f'Nom.{j}', f'Prénom.{j}', f'Voix.{j}', f'% Voix/Ins.{j}', f'% Voix/Exp.{j}']
        tableau = df[colonnes]
        tableau.columns=colonnes_a_garder
        nouveau_df = pd.concat([nouveau_df, tableau], ignore_index=True)


    # Ajout de la colonne 'Date' avec une année données
    nouveau_df['Date'] = pd.to_datetime(str(date), format='%Y').year
    # Réinitialisation de l'index
    nouveau_df.reset_index(drop=True, inplace=True)
    # Ajouter le nouveau DataFrame et la date correspondante à la liste
    nouveaux_dataframes.append(nouveau_df)

# Concaténer tous les nouveaux DataFrames en un seul DataFrame
resultat_final = pd.concat(nouveaux_dataframes, ignore_index=True)

# Afficher les premières lignes du DataFrame final
resultat_final

In [ ]:
import pandas as pd
import os
from tqdm import tqdm

# Liste des noms de fichiers Excel
fichiers_excel = [
    r"C:\Users\sebeo\OneDrive\Bureau\DATASET\tr 2\2002 tr 2.xls",
    r"C:\Users\sebeo\OneDrive\Bureau\DATASET\tr 2\2007 tr 2.xlsx",
    r"C:\Users\sebeo\OneDrive\Bureau\DATASET\tr 2\2012 tr 2.xlsx",
    r"C:\Users\sebeo\OneDrive\Bureau\DATASET\tr 2\2017 tr 2.xls",
    r"C:\Users\sebeo\OneDrive\Bureau\DATASET\tr 2\2022 tr 2.xlsx"
]

# Initialisation d'une liste pour stocker les nouveaux DataFrames et les dates correspondantes
nouveaux_dataframes = []
dates = [2002,2007,2012,2017,2022]

# Parcourir chaque fichier Excel
for fichier, date in zip(fichiers_excel, dates):

    # Charger le fichier Excel dans un DataFrame pandas
    df = pd.read_excel(fichier)

    # Colonnes à garder
    colonnes_a_garder = ['Libellé de la commune','Inscrits','Abstentions','Votants','Exprimés', 'Sexe', 'Nom', 'Prénom', 'Voix', '% Voix/Ins', '% Voix/Exp']

    # Création d'un nouveau DataFrame avec les colonnes sélectionnées
    nouveau_df = df[colonnes_a_garder].copy()

    # Ajout des valeurs des colonnes supplémentaires
    x = 2
    for j in tqdm(range(1, x)):
        colonnes = ['Libellé de la commune','Inscrits','Abstentions','Votants','Exprimés', f'Sexe.{j}', f'Nom.{j}', f'Prénom.{j}', f'Voix.{j}', f'% Voix/Ins.{j}', f'% Voix/Exp.{j}']
        tableau = df[colonnes]
        tableau.columns=colonnes_a_garder
        nouveau_df = pd.concat([nouveau_df, tableau], ignore_index=True)


    # Ajout de la colonne 'Date' avec une année données
    nouveau_df['Date'] = pd.to_datetime(str(date), format='%Y').year
    # Réinitialisation de l'index
    nouveau_df.reset_index(drop=True, inplace=True)
    # Ajouter le nouveau DataFrame et la date correspondante à la liste
    nouveaux_dataframes.append(nouveau_df)

# Concaténer tous les nouveaux DataFrames en un seul DataFrame
tour2_result = pd.concat(nouveaux_dataframes, ignore_index=True)

tour2_result.columns= ['Libellé de la commune', 'Inscrits2',
       'Abstentions2', 'Votants2', 'Exprimés2', 'Sexe', 'Nom', 'Prénom', 'Voix2',
       '% Voix/Ins2', '% Voix/Exp2', 'Date']
# Afficher les premières lignes du DataFrame final
tour2_result

In [ ]:
# Fusionner les datasets pour le tour 1 et celui du tour 2 en utilisant les colonnes spécifiées et en conservant toutes les lignes
elections= tour2_result.merge(resultat_final, on=['Libellé de la commune', 'Prénom', 'Nom', 'Sexe', 'Date'], how='outer')

# Remplacer les valeurs NaN par zéro
elections.fillna(0, inplace=True)

# Afficher les premières lignes du DataFrame fusionné
elections

#Fusion des Datasets déjà traités


In [ ]:
# Importer la bibliothèque pandas
import pandas as pd

# Définir les dates pour lesquelles vous souhaitez fusionner les données
dates = [2022]

# Filtrer les ensembles de données originaux pour les dates spécifiées
elections_filtered = elections[elections['Date'].isin(dates)]
niveau_education_filtered = Niveau_education[Niveau_education['Date'].isin(dates)]
revenu_final_filtered = revenu_final[revenu_final['Date'].isin(dates)]
demographie_filtered = Demographie[Demographie['Date'].isin(dates)]

# Fusionner les ensembles de données en utilisant les colonnes communes
Dataset = pd.merge(elections_filtered, niveau_education_filtered, on=['Libellé de la commune', 'Date']) \
             .merge(revenu_final_filtered, on=['Libellé de la commune', 'Date']) \
             .merge(demographie_filtered, on=['Libellé de la commune', 'Date'])

# Afficher le jeu de données fusionné
print(Dataset)


In [ ]:
"""# Fusionner les datasets en utilisant les colonnes communes 'Libellé de la commune' et 'Date'
Dataset = elections.merge(Niveau_education, on=['Libellé de la commune', 'Date']) \
                           .merge(revenu_final, on=['Libellé de la commune', 'Date']) \
                           .merge(Demographie, on=['Libellé de la commune', 'Date'])

# Afficher la dataset fusionné
Dataset"""

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
encoder=LabelEncoder()

# Créer une nouvelle colonne en concaténant les colonnes 'Nom' et 'Prénom'
Dataset['Nom_complet'] = Dataset['Nom'] + ' ' + Dataset['Prénom']


# Effectuer l'encodage des étiquettes dans la colonne 'Nom_complet'
Dataset['Nom_complet']= le.fit_transform(Dataset['Nom_complet'])

Dataset['Libellé de la commune']= encoder.fit_transform(Dataset['Libellé de la commune'])

# Supprimer les colonnes 'Nom' et 'Prénom'
Dataset.drop(['Nom', 'Prénom'], axis=1, inplace=True)

##realisation du one-hot encoding pour les colonnes les features en chaine de caractères

# Ajouter les colonnes  à la liste des colonnes à encoder en one-hot
one_hot_encode_cols = ['Sexe']

# Appliquer le one-hot encoding aux colonnes sélectionnées
Dataset = pd.get_dummies(Dataset, columns=one_hot_encode_cols, drop_first=True)

In [ ]:
# Transformation inverse de toutes les valeurs encodées dans la colonne 'Nom_complet'
decoded_values = le.inverse_transform(Dataset['Nom_complet'])

# Affichage des valeurs décodées
print(decoded_values)

In [ ]:
# Créer une nouvelle colonne 'President' remplie de par prdant pour les candidats avec des voix inferieur a celui du vainqueur
Dataset['President'] = "perdant"

# Parcourir chaque date unique dans la colonne 'Date'
for date in Dataset['Date'].unique():
    # Parcourir chaque commune unique dans le dataset
    for commune in Dataset['Libellé de la commune'].unique():
        # Sélectionner les lignes correspondant à cette date et cette commune
        subset = Dataset[(Dataset['Date'] == date) & (Dataset['Libellé de la commune'] == commune)]
        # Trouver la valeur maximale dans la colonne 'Voix2' pour cette date et cette commune
        max_value = subset['Voix2'].max()
        # Mettre "vainqueur" dans la colonne 'President' pour les lignes ayant la valeur maximale
        Dataset.loc[(Dataset['Date'] == date) & (Dataset['Libellé de la commune'] == commune) & (Dataset['Voix2'] == max_value), 'President'] = "vainqueur"
        # Mettre "perdant" dans la colonne 'President' pour les lignes n'ayant pas la valeur maximale
        Dataset.loc[(Dataset['Date'] == date) & (Dataset['Libellé de la commune'] == commune) & (Dataset['Voix2'] != max_value), 'President'] = "perdant"


In [ ]:
le=LabelEncoder()
# Effectuer l'encodage des étiquettes dans la colonne 'Attack_type'
Dataset["President"] = le.fit_transform(Dataset["President"])

#Nettoyage des données

In [ ]:
# Supprimer les colonnes 'Nom' et 'Prénom'
Dataset.drop(['% Voix/Ins2', '% Voix/Exp2','Voix2'], axis=1, inplace=True)

#Préparation des vécteurs
X = Dataset.loc[:, "Libellé de la commune":"President"].drop(columns=["President"])
Y=Dataset["President"]

In [ ]:
le=LabelEncoder()
# Effectuer l'encodage des étiquettes dans la colonne 'Attack_type'
Y = le.fit_transform(Y)

In [ ]:
unique_values_Y = Dataset["President"].unique()
print(unique_values_Y)


In [ ]:
# Importer la bibliothèque pandas
import pandas as pd

# Calculer le décompte pour chaque catégorie dans la colonne "President"
count_per_category = Dataset["President"].value_counts()

# Afficher le résultat
print(count_per_category)


In [ ]:
from sklearn.preprocessing import StandardScaler
# Appliquer le StandardScaler aux caractéristiques numériques
scaler = StandardScaler()
X_standardiser = scaler.fit_transform(X)
print(X.shape)

In [ ]:
import seaborn as sns

# Supposons que X_standardiser est notre tableau NumPy contenant les données standardisées

# Convertir X_standardiser en un DataFrame pandas avec des colonnes appropriées
df_standardise = pd.DataFrame(X_standardiser)

# Calculer la corrélation entre les variables
corr = df_standardise.corr()

# Tracer la heatmap de la corrélation
sns.heatmap(data=corr)

In [ ]:
from sklearn.decomposition import PCA

import pandas as pd

# Création de l'objet PCA avec un nombre de composants approprié
pca = PCA(n_components=15, svd_solver='full', random_state=1001)

# Ajustement du PCA sur les données standardisées
X_pca = pca.fit_transform(df_standardise)

# Obtenez les noms des entités en sortie pour la transformation
feature_names_out = [f"PC{j+1}" for j in range(X_pca.shape[1])]  # Créez les noms des composants principaux

# Créer un DataFrame pandas avec les données transformées
data_pca = pd.DataFrame(X_pca, columns=feature_names_out)

# Afficher les données PCA
print(data_pca)


In [ ]:
print('Explained variance: %.4f' % pca.explained_variance_ratio_.sum())

In [ ]:
# Importer les bibliothèques nécessaires
from sklearn.model_selection import train_test_split


# Diviser les données en ensembles d'apprentissage et de test
X_train, X_test, y_train, y_test = train_test_split(data_pca, Y, test_size=0.3, random_state=42)

#Arbre de décision

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
# modèle de Regression Logistique
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))

#Rapport de classification
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Supposez que y_true sont les vraies étiquettes et y_pred sont les prédictions du modèle
# Calculer la matrice de confusion
cm = confusion_matrix(y_test, y_pred)

# Afficher la matrice de confusion sous forme graphique
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='g')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()




#Regression Logistique

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
# modèle de Regression Logistique
model = LogisticRegression()
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))

#Rapport de classification
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Supposez que y_true sont les vraies étiquettes et y_pred sont les prédictions du modèle
# Calculer la matrice de confusion
cm = confusion_matrix(y_test, y_pred)

# Afficher la matrice de confusion sous forme graphique
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='g')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

#KNN

In [ ]:
import tensorflow as tf
from sklearn.metrics import classification_report

# Définition du modèle ANN
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compilation du modèle
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Entraînement du modèle
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Évaluation du modèle
y_pred_proba = model.predict(X_test)
y_pred = (y_pred_proba > 0.5).astype(int)

# Calcul de l'exactitude
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))

# Rapport de classification
print(classification_report(y_test, y_pred))


In [ ]:
# Supposez que y_true sont les vraies étiquettes et y_pred sont les prédictions du modèle
# Calculer la matrice de confusion
cm = confusion_matrix(y_test, y_pred)

# Afficher la matrice de confusion sous forme graphique
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='g')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()